In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols

from utils import (
    generatePEDModelsSkuStore,
    generatePEDModelsByPriceSkuStore
)

import warnings
warnings.filterwarnings("ignore")

In [2]:
df1 = pd.read_csv('SALES-full.csv', sep=';')

In [3]:
df1['curr_date'] = pd.to_datetime(df1['curr_date'])

# SKU|Store

In [4]:
index = ['SKU_id', 'Store_id']

In [5]:
df = df1[0:40000000]
df.shape

(40000000, 9)

In [6]:
df.rename(columns={'Code_AP' : 'SKU_id',
                   'store_id' : 'Store_id',
                   'curr_date' : 'Date',
                   'stock' : 'Stock',
                   'sales_regular' : 'Sales',
                   'price_regular' : 'Regular_Price'
                    }, inplace=True)
df.drop(['sales_promo', 'price_promo', 'flag_promo'], axis=1, inplace=True)

In [7]:
df = df.loc[df['Stock'] > df['Sales']]
df['Demand'] = df.loc[df['Stock'] > 0, 'Sales']
df.set_index(['SKU_id', 'Store_id', 'Date'], inplace=True)

In [8]:
level_values = df.index.get_level_values
df_w = (df.groupby([level_values(i) for i in [0,1]]
                      +[pd.Grouper(freq='W', level=-1)]).mean())
# агрегация исходных данные в разрезе SKU|Store|Week

In [9]:
df_w.reset_index(inplace=True)
df.reset_index(inplace=True)

In [10]:
df['Round_Price'] = round(df['Regular_Price'])
df_w['Round_Price'] = round(df_w['Regular_Price'])

In [11]:
popular_index = df.groupby(index).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
num_day = popular_index[popular_index['counts'] >= 1000].shape[0]
popular_index[popular_index['counts'] >= 1000]

,SKU_id,Store_id,counts
0,5606,2834,1088
1,13316,2834,1088
2,3414,2834,1088
3,4404,2834,1088
4,1823,2834,1088
...,...,...,...
11745,1358,2231,1000
11746,11916,1128,1000
11747,7907,1900,1000
11748,1827,1079,1000


In [12]:
popular_pairs = df_w.groupby(index).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
num_week = popular_pairs[popular_pairs['counts'] >= 150].shape[0]
popular_pairs[popular_pairs['counts'] >= 150]

,SKU_id,Store_id,counts
0,10220,2834,156
1,14228,1126,156
2,3045,1111,156
3,12586,1124,156
4,9582,1085,156
...,...,...,...
12044,1911,1138,150
12045,12582,1841,150
12046,1952,1173,150
12047,5454,1136,150


In [13]:
sample_data_day_0to40 = generatePEDModelsSkuStore(df, amount=num_day)
sample_data_day_0to40['Гранулярность эластичности'] = 'SKU|Store'
sample_data_day_0to40['Гранулярность trainset'] = 'SKU|Store|Day'

In [14]:
sample_data_day_0to40_p = generatePEDModelsByPriceSkuStore(df, amount=num_day)
sample_data_day_0to40_p['Гранулярность эластичности'] = 'SKU|Store'
sample_data_day_0to40_p['Гранулярность trainset'] = 'SKU|Store|Day'

In [15]:
sample_data_week_0to40 = generatePEDModelsSkuStore(df_w, date_type='week', amount=num_week)
sample_data_week_0to40['Гранулярность эластичности'] = 'SKU|Store'
sample_data_week_0to40['Гранулярность trainset'] = 'SKU|Store|Week'

In [16]:
sample_data_week_0to40_p = generatePEDModelsByPriceSkuStore(df_w, date_type='week', amount=num_week)
sample_data_week_0to40_p['Гранулярность эластичности'] = 'SKU|Store'
sample_data_week_0to40_p['Гранулярность trainset'] = 'SKU|Store|Week'

In [17]:
df = df1[40000000:80000000]
df.shape

(40000000, 9)

In [18]:
df.rename(columns={'Code_AP' : 'SKU_id',
                   'store_id' : 'Store_id',
                   'curr_date' : 'Date',
                   'stock' : 'Stock',
                   'sales_regular' : 'Sales',
                   'price_regular' : 'Regular_Price'
                    }, inplace=True)
df.drop(['sales_promo', 'price_promo', 'flag_promo'], axis=1, inplace=True)

In [19]:
df = df.loc[df['Stock'] > df['Sales']]
df['Demand'] = df.loc[df['Stock'] > 0, 'Sales']
df.set_index(['SKU_id', 'Store_id', 'Date'], inplace=True)

In [20]:
level_values = df.index.get_level_values
df_w = (df.groupby([level_values(i) for i in [0,1]]
                      +[pd.Grouper(freq='W', level=-1)]).mean())
# агрегация исходных данные в разрезе SKU|Store|Week

In [21]:
df_w.reset_index(inplace=True)
df.reset_index(inplace=True)

In [22]:
df['Round_Price'] = round(df['Regular_Price'])
df_w['Round_Price'] = round(df_w['Regular_Price'])

In [23]:
popular_pairs = df.groupby(['SKU_id', 'Store_id']).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
num_day = popular_pairs[popular_pairs['counts'] >= 1000].shape[0]
popular_pairs[popular_pairs['counts'] >= 1000]

,SKU_id,Store_id,counts
0,20724,2834,1088
1,18469,2834,1088
2,89273,2834,1088
3,15775,2834,1088
4,76690,2834,1088
...,...,...,...
11031,60656,1095,1000
11032,90156,1123,1000
11033,15665,1127,1000
11034,17221,1123,1000


In [24]:
popular_pairs = df_w.groupby(['SKU_id', 'Store_id']).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
num_week = popular_pairs[popular_pairs['counts'] >= 150].shape[0]
popular_pairs[popular_pairs['counts'] >= 150]

,SKU_id,Store_id,counts
0,76687,1117,156
1,20280,1106,156
2,16260,1111,156
3,16260,1112,156
4,16260,1113,156
...,...,...,...
11477,20617,1138,150
11478,90976,1101,150
11479,32430,1110,150
11480,20617,1137,150


In [25]:
sample_data_day_40to80 = generatePEDModelsSkuStore(df, amount=num_day)
sample_data_day_40to80['Гранулярность эластичности'] = 'SKU|Store'
sample_data_day_40to80['Гранулярность trainset'] = 'SKU|Store|Day'

In [26]:
sample_data_day_40to80_p = generatePEDModelsByPriceSkuStore(df, amount=num_day)
sample_data_day_40to80_p['Гранулярность эластичности'] = 'SKU|Store'
sample_data_day_40to80_p['Гранулярность trainset'] = 'SKU|Store|Day'

In [27]:
sample_data_week_40to80 = generatePEDModelsSkuStore(df_w, date_type='week', amount=num_week)
sample_data_week_40to80['Гранулярность эластичности'] = 'SKU|Store'
sample_data_week_40to80['Гранулярность trainset'] = 'SKU|Store|Week'

In [28]:
sample_data_week_40to80_p = generatePEDModelsByPriceSkuStore(df_w, date_type='week', amount=num_week)
sample_data_week_40to80_p['Гранулярность эластичности'] = 'SKU|Store'
sample_data_week_40to80_p['Гранулярность trainset'] = 'SKU|Store|Week'

In [29]:
df = df1[80000000:120000000]
df.shape

(40000000, 9)

In [30]:
df.rename(columns={'Code_AP' : 'SKU_id',
                   'store_id' : 'Store_id',
                   'curr_date' : 'Date',
                   'stock' : 'Stock',
                   'sales_regular' : 'Sales',
                   'price_regular' : 'Regular_Price'
                    }, inplace=True)
df.drop(['sales_promo', 'price_promo', 'flag_promo'], axis=1, inplace=True)

In [31]:
df = df.loc[df['Stock'] > df['Sales']]
df['Demand'] = df.loc[df['Stock'] > 0, 'Sales']
df.set_index(['SKU_id', 'Store_id', 'Date'], inplace=True)

In [32]:
level_values = df.index.get_level_values
df_w = (df.groupby([level_values(i) for i in [0,1]]
                      +[pd.Grouper(freq='W', level=-1)]).mean())
# агрегация исходных данные в разрезе SKU|Store|Week

In [33]:
df_w.reset_index(inplace=True)
df.reset_index(inplace=True)

In [34]:
df['Round_Price'] = round(df['Regular_Price'])
df_w['Round_Price'] = round(df_w['Regular_Price'])

In [35]:
popular_pairs = df.groupby(['SKU_id', 'Store_id']).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
num_day = popular_pairs[popular_pairs['counts'] >= 1000].shape[0]
popular_pairs[popular_pairs['counts'] >= 1000]

,SKU_id,Store_id,counts
0,98567,2834,1088
1,95574,2834,1088
2,99251,2834,1088
3,112073,2834,1088
4,118696,2834,1088
...,...,...,...
10621,95752,1950,1000
10622,104086,1111,1000
10623,98677,1201,1000
10624,118244,1182,1000


In [36]:
popular_pairs = df_w.groupby(['SKU_id', 'Store_id']).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
num_week = popular_pairs[popular_pairs['counts'] >= 150].shape[0]
popular_pairs[popular_pairs['counts'] >= 150]

,SKU_id,Store_id,counts
0,119173,1077,156
1,103496,1106,156
2,103495,1093,156
3,103495,1095,156
4,103495,1097,156
...,...,...,...
11014,105272,1118,150
11015,96261,1170,150
11016,119545,1137,150
11017,96261,1189,150


In [37]:
sample_data_day_80to120 = generatePEDModelsSkuStore(df, amount=num_day)
sample_data_day_80to120['Гранулярность эластичности'] = 'SKU|Store'
sample_data_day_80to120['Гранулярность trainset'] = 'SKU|Store|Day'

In [38]:
sample_data_day_80to120_p = generatePEDModelsByPriceSkuStore(df, amount=num_day)
sample_data_day_80to120_p['Гранулярность эластичности'] = 'SKU|Store'
sample_data_day_80to120_p['Гранулярность trainset'] = 'SKU|Store|Day'

In [39]:
sample_data_week_80to120 = generatePEDModelsSkuStore(df_w, date_type='week', amount=num_week)
sample_data_week_80to120['Гранулярность эластичности'] = 'SKU|Store'
sample_data_week_80to120['Гранулярность trainset'] = 'SKU|Store|Week'

In [40]:
sample_data_week_80to120_p = generatePEDModelsByPriceSkuStore(df_w, date_type='week', amount=num_week)
sample_data_week_80to120_p['Гранулярность эластичности'] = 'SKU|Store'
sample_data_week_80to120_p['Гранулярность trainset'] = 'SKU|Store|Week'

In [41]:
df = df1[120000000:160000000]
df.shape

(40000000, 9)

In [42]:
df.rename(columns={'Code_AP' : 'SKU_id',
                   'store_id' : 'Store_id',
                   'curr_date' : 'Date',
                   'stock' : 'Stock',
                   'sales_regular' : 'Sales',
                   'price_regular' : 'Regular_Price'
                    }, inplace=True)
df.drop(['sales_promo', 'price_promo', 'flag_promo'], axis=1, inplace=True)

In [43]:
df = df.loc[df['Stock'] > df['Sales']]
df['Demand'] = df.loc[df['Stock'] > 0, 'Sales']
df.set_index(['SKU_id', 'Store_id', 'Date'], inplace=True)

In [44]:
level_values = df.index.get_level_values
df_w = (df.groupby([level_values(i) for i in [0,1]]
                      +[pd.Grouper(freq='W', level=-1)]).mean())
# агрегация исходных данные в разрезе SKU|Store|Week

In [45]:
df_w.reset_index(inplace=True)
df.reset_index(inplace=True)

In [46]:
df['Round_Price'] = round(df['Regular_Price'])
df_w['Round_Price'] = round(df_w['Regular_Price'])

In [47]:
popular_pairs = df.groupby(['SKU_id', 'Store_id']).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
num_day = popular_pairs[popular_pairs['counts'] >= 1000].shape[0]
popular_pairs[popular_pairs['counts'] >= 1000]

,SKU_id,Store_id,counts
0,126376,2834,1088
1,131781,2834,1088
2,129979,2834,1088
3,124063,2834,1088
4,141821,2834,1088
...,...,...,...
7531,140338,1136,1000
7532,134991,1881,1000
7533,134247,1780,1000
7534,130202,1090,1000


In [48]:
popular_pairs = df_w.groupby(['SKU_id', 'Store_id']).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
num_week = popular_pairs[popular_pairs['counts'] >= 150].shape[0]
popular_pairs[popular_pairs['counts'] >= 150]

,SKU_id,Store_id,counts
0,131732,1112,156
1,140313,1103,156
2,140313,1111,156
3,140313,1109,156
4,140313,1107,156
...,...,...,...
7870,139210,1123,150
7871,132376,1138,150
7872,126272,1138,150
7873,135016,1168,150


In [49]:
sample_data_day_120to160 = generatePEDModelsSkuStore(df, amount=num_day)
sample_data_day_120to160['Гранулярность эластичности'] = 'SKU|Store'
sample_data_day_120to160['Гранулярность trainset'] = 'SKU|Store|Day'

In [50]:
sample_data_day_120to160_p = generatePEDModelsByPriceSkuStore(df, amount=num_day)
sample_data_day_120to160_p['Гранулярность эластичности'] = 'SKU|Store'
sample_data_day_120to160_p['Гранулярность trainset'] = 'SKU|Store|Day'

In [51]:
sample_data_week_120to160 = generatePEDModelsSkuStore(df_w, date_type='week', amount=num_week)
sample_data_week_120to160['Гранулярность эластичности'] = 'SKU|Store'
sample_data_week_120to160['Гранулярность trainset'] = 'SKU|Store|Week'

In [52]:
sample_data_week_120to160_p = generatePEDModelsByPriceSkuStore(df_w, date_type='week', amount=num_week)
sample_data_week_120to160_p['Гранулярность эластичности'] = 'SKU|Store'
sample_data_week_120to160_p['Гранулярность trainset'] = 'SKU|Store|Week'

In [53]:
df = df1[160000000:183051540]
df.shape

(23051540, 9)

In [54]:
df.rename(columns={'Code_AP' : 'SKU_id',
                   'store_id' : 'Store_id',
                   'curr_date' : 'Date',
                   'stock' : 'Stock',
                   'sales_regular' : 'Sales',
                   'price_regular' : 'Regular_Price'
                    }, inplace=True)
df.drop(['sales_promo', 'price_promo', 'flag_promo'], axis=1, inplace=True)

In [55]:
df = df.loc[df['Stock'] > df['Sales']]
df['Demand'] = df.loc[df['Stock'] > 0, 'Sales']
df.set_index(['SKU_id', 'Store_id', 'Date'], inplace=True)

In [56]:
level_values = df.index.get_level_values
df_w = (df.groupby([level_values(i) for i in [0,1]]
                      +[pd.Grouper(freq='W', level=-1)]).mean())
# агрегация исходных данные в разрезе SKU|Store|Week

In [57]:
df_w.reset_index(inplace=True)
df.reset_index(inplace=True)

In [58]:
df['Round_Price'] = round(df['Regular_Price'])
df_w['Round_Price'] = round(df_w['Regular_Price'])

In [59]:
popular_pairs = df.groupby(['SKU_id', 'Store_id']).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
num_day = popular_pairs[popular_pairs['counts'] >= 1000].shape[0]
popular_pairs[popular_pairs['counts'] >= 1000]

,SKU_id,Store_id,counts
0,317998,2834,1088
1,211494,2834,1088
2,325632,2834,1088
3,149856,2834,1088
4,363195,2834,1088
...,...,...,...
662,145864,1136,1000
663,146047,1111,1000
664,325632,1108,1000
665,145864,1084,1000


In [60]:
popular_pairs = df_w.groupby(['SKU_id', 'Store_id']).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
num_week = popular_pairs[popular_pairs['counts'] >= 150].shape[0]
popular_pairs[popular_pairs['counts'] >= 150]

,SKU_id,Store_id,counts
0,325632,1123,156
1,363196,1085,156
2,363196,1091,156
3,363195,1098,156
4,363196,1090,156
...,...,...,...
581,211494,1113,150
582,145864,1115,150
583,325632,1076,150
584,149857,1896,150


In [61]:
sample_data_day_160to_end = generatePEDModelsSkuStore(df, amount=num_day)
sample_data_day_160to_end['Гранулярность эластичности'] = 'SKU|Store'
sample_data_day_160to_end['Гранулярность trainset'] = 'SKU|Store|Day'

In [62]:
sample_data_day_160to_end_p = generatePEDModelsByPriceSkuStore(df, amount=num_day)
sample_data_day_160to_end_p['Гранулярность эластичности'] = 'SKU|Store'
sample_data_day_160to_end_p['Гранулярность trainset'] = 'SKU|Store|Day'

In [63]:
sample_data_week_160to_end = generatePEDModelsSkuStore(df_w, date_type='week', amount=num_week)
sample_data_week_160to_end['Гранулярность эластичности'] = 'SKU|Store'
sample_data_week_160to_end['Гранулярность trainset'] = 'SKU|Store|Week'

In [64]:
sample_data_week_160to_end_p = generatePEDModelsByPriceSkuStore(df_w, date_type='week', amount=num_week)
sample_data_week_160to_end_p['Гранулярность эластичности'] = 'SKU|Store'
sample_data_week_160to_end_p['Гранулярность trainset'] = 'SKU|Store|Week'

In [65]:
sample_data_day = sample_data_day_0to40.append([sample_data_day_40to80, sample_data_day_80to120, sample_data_day_120to160, sample_data_day_160to_end])
sample_data_day.reset_index(inplace=True, drop=True)
sample_data_day.to_csv('models_sku_store/sample_data_day.csv', index=False)
sample_data_day

,SKU,Store,Coefficient,R-squared,T-test-p-value,Model-type,Elasticity,Qty,Num_diff_prices,SampleSize,Гранулярность эластичности,Гранулярность trainset
0,5606,2834,1.173111,2.316427e-01,1.590257e-01,log-log,1.173111,2.0,9,10,SKU|Store,SKU|Store|Day
1,5606,2834,1.114628,1.382282e-01,1.065026e-01,log-log,1.114628,6.0,18,20,SKU|Store,SKU|Store|Day
2,5606,2834,0.528576,4.509032e-02,2.599342e-01,log-log,0.528576,6.0,25,30,SKU|Store,SKU|Store|Day
3,5606,2834,0.399299,2.042320e-02,3.790178e-01,log-log,0.399299,12.0,31,40,SKU|Store,SKU|Store|Day
4,5606,2834,0.002598,8.148037e-07,9.950361e-01,log-log,0.002598,15.0,35,50,SKU|Store,SKU|Store|Day
...,...,...,...,...,...,...,...,...,...,...,...,...
730722,211503,1111,-0.566334,2.944716e-02,1.151780e-04,log-log,-0.566334,45.0,13,500,SKU|Store,SKU|Store|Day
730723,211503,1111,-0.574718,3.117594e-02,1.360151e-05,log-log,-0.574718,52.0,14,600,SKU|Store,SKU|Store|Day
730724,211503,1111,-0.580943,3.049713e-02,3.353429e-06,log-log,-0.580943,64.0,14,700,SKU|Store,SKU|Store|Day
730725,211503,1111,-0.611979,3.518078e-02,9.073871e-08,log-log,-0.611979,70.0,14,800,SKU|Store,SKU|Store|Day


In [66]:
sample_data_day_p = sample_data_day_0to40_p.append([sample_data_day_40to80_p, sample_data_day_80to120_p, sample_data_day_120to160_p, sample_data_day_160to_end_p])
sample_data_day_p.reset_index(inplace=True, drop=True)
sample_data_day_p.to_csv('models_sku_store/sample_data_day_p.csv', index=False)
sample_data_day_p

,SKU,Store,Coefficient,R-squared,T-test-p-value,Model-type,Elasticity,Qty,SampleSize,Гранулярность эластичности,Гранулярность trainset
0,5606,2834,1.040991,0.293878,8.394226e-08,log-log,1.040991,29.0,2,SKU|Store,SKU|Store|Day
1,5606,2834,1.120356,0.225558,2.193155e-08,log-log,1.120356,55.0,3,SKU|Store,SKU|Store|Day
2,5606,2834,1.146144,0.174323,3.246592e-08,log-log,1.146144,96.0,4,SKU|Store,SKU|Store|Day
3,5606,2834,1.290883,0.269283,5.636590e-15,log-log,1.290883,96.0,5,SKU|Store,SKU|Store|Day
4,5606,2834,0.889247,0.127720,2.839535e-08,log-log,0.889247,124.0,6,SKU|Store,SKU|Store|Day
...,...,...,...,...,...,...,...,...,...,...,...
1120197,211503,1111,-0.767817,0.056524,5.545838e-14,log-log,-0.767817,83.0,10,SKU|Store,SKU|Store|Day
1120198,211503,1111,-0.683252,0.042713,4.958414e-11,log-log,-0.683252,90.0,11,SKU|Store,SKU|Store|Day
1120199,211503,1111,-0.683632,0.043094,3.623138e-11,log-log,-0.683632,90.0,12,SKU|Store,SKU|Store|Day
1120200,211503,1111,-0.683736,0.043104,3.523037e-11,log-log,-0.683736,90.0,13,SKU|Store,SKU|Store|Day


In [67]:
sample_data_week = sample_data_week_0to40.append([sample_data_week_40to80, sample_data_week_80to120, sample_data_week_120to160, sample_data_week_160to_end])
sample_data_week.reset_index(inplace=True, drop=True)
sample_data_week.to_csv('models_sku_store/sample_data_week.csv', index=False)
sample_data_week

,SKU,Store,Coefficient,R-squared,T-test-p-value,Model-type,Elasticity,Qty,Num_diff_prices,SampleSize,Гранулярность эластичности,Гранулярность trainset
0,10220,2834,-6.467575,0.315375,0.091158,log-log,-6.467575,7.857143,8,10,SKU|Store,SKU|Store|Week
1,10220,2834,1.663427,0.017660,0.576489,log-log,1.663427,16.285714,11,20,SKU|Store,SKU|Store|Week
2,10220,2834,2.758622,0.055194,0.211415,log-log,2.758622,18.285714,12,30,SKU|Store,SKU|Store|Week
3,10220,2834,3.506491,0.083252,0.070981,log-log,3.506491,25.000000,13,40,SKU|Store,SKU|Store|Week
4,10220,2834,2.223723,0.035826,0.188012,log-log,2.223723,32.428571,15,50,SKU|Store,SKU|Store|Week
...,...,...,...,...,...,...,...,...,...,...,...,...
632697,146728,1099,-2.383424,0.051501,0.023174,log-log,-2.383424,134.152381,50,100,SKU|Store,SKU|Store|Week
632698,146728,1099,-1.977179,0.037591,0.042402,log-log,-1.977179,148.842857,53,110,SKU|Store,SKU|Store|Week
632699,146728,1099,-1.985818,0.037278,0.034616,log-log,-1.985818,155.152381,55,120,SKU|Store,SKU|Store|Week
632700,146728,1099,-1.755337,0.028907,0.053124,log-log,-1.755337,164.323810,55,130,SKU|Store,SKU|Store|Week


In [68]:
sample_data_week_p = sample_data_week_0to40_p.append([sample_data_week_40to80_p, sample_data_week_80to120_p, sample_data_week_120to160_p, sample_data_week_160to_end_p])
sample_data_week_p.reset_index(inplace=True, drop=True)
sample_data_week_p.to_csv('models_sku_store/sample_data_week_p.csv', index=False)
sample_data_week_p

,SKU,Store,Coefficient,R-squared,T-test-p-value,Model-type,Elasticity,Qty,SampleSize,Гранулярность эластичности,Гранулярность trainset
0,10220,2834,6.990365,0.527844,2.365041e-08,log-log,6.990365,13.952381,2,SKU|Store,SKU|Store|Week
1,10220,2834,7.118371,0.337789,9.051607e-07,log-log,7.118371,22.952381,3,SKU|Store,SKU|Store|Week
2,10220,2834,7.843127,0.284701,5.832664e-07,log-log,7.843127,37.095238,4,SKU|Store,SKU|Store|Week
3,10220,2834,8.691006,0.394964,1.538554e-11,log-log,8.691006,58.380952,5,SKU|Store,SKU|Store|Week
4,10220,2834,7.969932,0.334553,5.538312e-11,log-log,7.969932,69.809524,6,SKU|Store,SKU|Store|Week
...,...,...,...,...,...,...,...,...,...,...,...
1462589,146728,1099,-1.575043,0.023867,6.354517e-02,log-log,-1.575043,182.300000,56,SKU|Store,SKU|Store|Week
1462590,146728,1099,-1.564860,0.023354,6.554987e-02,log-log,-1.564860,184.871429,57,SKU|Store,SKU|Store|Week
1462591,146728,1099,-1.568985,0.023449,6.407013e-02,log-log,-1.568985,185.442857,58,SKU|Store,SKU|Store|Week
1462592,146728,1099,-1.513058,0.021605,7.463665e-02,log-log,-1.513058,185.442857,59,SKU|Store,SKU|Store|Week
